<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Telecomm EDA Challenge Lab

_Author: Alex Combs (NYC) _

---

Let's do some Exploratory Data Analysis (EDA)! As a data scientist, you often may find yourself given a data set you've never seen before, and asked to do a rapid analysis. This is today's goal.

# Prompt

You work for a telecommunications company. The company has been storing metadata about customer phone usage, as part of the regular course of business. Currently, this data is sitting in an unsecured database. The company doesn't want to pay to increase their database security, because they don't think there's really anything to be learned from the metadata.

They are under pressure from "right to privacy" organizations to beef up the database security. These organizations argue that you can learn a lot about a person from their cell phone metadata.

The telecom company wants to understand if this is true, and they want your help. They will give you one person's metadata for 2014 and want to see what you can learn from it.

Working in teams, create a report revealing everything you can about the person. Prepare a presentation, with slides, showcasing your findings.


# The Data

The [person's metadata](./datasets/metadata.csv) has the following fields:

| Field Name          | Description
| ---                 | ---
| **Cell Cgi**        | cell phone tower identifier
| **Cell Tower**      | cell phone tower location
| **Comm Identifier** |	de-identified recipient of communication
| **Comm Timedate String** | time of communication
| **Comm Type	Id**  | type of communication
| **Latitude**        | latitude of communication
| **Longitude**       | longitude of communication


# Hints

This is totally open-ended! If you're totally stumped -- and only if stumped -- should you look below for prompts. As a starting point, given that you have geo-locations, consider investigating ways to display this type of information (i.e. mapping functionality).

<font color='white'>
Well for starters, he's in Australia!

Ideas for things to look into:
- where does he work?
- where does he live?
- who does he contact most often?
- what hours does he work?
- did he move?
- did he go on holiday?  If so, where did he go?
- did he get a new phone?

Challenges:
- how does he get to work?
- where does his family live?
- if he went on holiday, can you find which flights he took?
- can you guess who some of his contacts are, based on the frequency, location, time and mode (phone/text) of communications?


If you're stuck on how to map the data, you can try "basemap" or "gmplot", or anything else you find online.
</font>

### Summary of Findings:
- We can assume based on telecomm data that this individual has been in Australia from April 2014 - 2015
- Data usage primarily in New South Wales, Victoria, and Tasmania, Australia
- This user primarily used Internet (~90%) communication type data compared to Phone and SMS
- Based on heatmap, we assume that this person probably lives in Sydney and travels to Melbourne and Tasmania a lot
- Most of time within Manly and Downtown Sydney

## General Data info

In [163]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import gpxpy.geo

%matplotlib inline

In [164]:
df = pd.read_csv('/Users/edwardhuang/Desktop/DS-SF-42/classes/09/eda-telecomm_group_project-lab/datasets/metadata.csv')

In [165]:
df.head()

,Cell Cgi,Cell Tower Location,Comm Identifier,Comm Timedate String,Comm Type,Latitude,Longitude
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296
1,50501015388B9,REDFERN TE,62157ccf2910019ffd915b11fa037243b75c1624,4/1/14 9:42,Phone,-33.892933,151.202296
2,505010153111F,HAYMARKET #,c8f92bd0f4e6fb45ed7fce96fc831b283db2b642,4/1/14 13:13,Phone,-33.880329,151.205690
3,505010153111F,HAYMARKET #,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 13:13,Phone,-33.880329,151.205690
4,5.05E+106,HAYMARKET #,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 17:27,Phone,-33.880329,151.205690


In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10476 entries, 0 to 10475
Data columns (total 7 columns):
Cell Cgi                10476 non-null object
Cell Tower Location     10476 non-null object
Comm Identifier         1374 non-null object
Comm Timedate String    10476 non-null object
Comm Type               10476 non-null object
Latitude                10476 non-null float64
Longitude               10476 non-null float64
dtypes: float64(2), object(5)
memory usage: 573.0+ KB


### Why does Comm Identifier have so many nulls?

In [167]:
null = df[df['Comm Identifier'].isnull()]

In [168]:
null['Comm Type'].unique()

array(['Internet'], dtype=object)

In [169]:
notnull = df[df['Comm Identifier'].notnull()]

In [170]:
notnull['Comm Type'].unique()

array(['Phone', 'SMS'], dtype=object)

### Discovered user communication data mainly for "Internet" type instead of "Phone" and "SMS"

### How many Cell Towers are there, and what's the most frequently used one?

In [171]:
df['Cell Tower Location'].value_counts()

BALGOWLAH HAYES ST                          4301
CHIPPENDALE                                 1084
SUNDERLAND ST                                723
REDFERN TE                                   712
HAYMARKET #                                  563
BRICKWORKS                                   501
HARBORD 22 WAINE ST                          465
FAIRLIGHT 137 SYDNEY RD                      454
MANLY #                                      231
NEW TOWN                                     197
CHINATOWN                                    161
BEECHWORTH                                   112
BALGOWLAH VILLAGE SHOPPING CENTRE IBC        106
MANLY SOUTH STEYNE                            92
BROADWAY OTC                                  85
MASCOT INTERNATIONAL AIRPORT TERMINAL T1      65
71 MACQUARIE ST                               49
SURRY HILLS 418A ELIZABETH ST                 45
MANLY NTH STEYNE                              40
MASCOT M5 MOTORWAY EMERGENCY STAIRS           33
BALGOWLAH TE        

Answer: There are 71 towers, BALGOWLAH HAYES ST is the most frequently used on

### Whats the timeframe of this data?

In [172]:
df['Comm Timedate String'].max()

'9/8/14 8:17'

In [173]:
pd.to_datetime(df['Comm Timedate String']).describe()

count                   10476
unique                   6382
top       2015-03-31 18:59:00
freq                       26
first     2014-04-01 09:40:00
last      2015-04-01 23:49:00
Name: Comm Timedate String, dtype: object

Answer: 4/1/14 - 4/1/15

In [174]:
print(df['Cell Tower Location'].unique().shape)
print(df['Latitude'].unique().shape)
print(df['Longitude'].unique().shape)

(71,)
(70,)
(70,)


Build a map_1 with all the towers on icons

In [175]:
map_1 = folium.Map(location=[-33.892933,151.202296],
                   zoom_start=4)

In [176]:
folium.Marker([-33.880329,151.205690],
              popup='Tower',
              icon=folium.Icon(color='red',icon='info-sign')
              ).add_to(map_1)

In [177]:
a = df['Latitude'].unique()
b = df['Longitude'].unique()
c = zip(a, b)


In [178]:
for x in c:
    folium.Marker(x,
              popup='Tower',
              icon=folium.Icon(color='red',icon='info-sign')
              ).add_to(map_1)
map_1

In [ ]:
map_1

In [179]:
d = zip(df['Latitude'], df['Longitude'])

In [180]:
from folium.plugins import HeatMap

In [181]:
map_2 = folium.Map(location=[-33.892933,151.202296],
                   zoom_start=4)

In [182]:
HeatMap(d).add_to(map_2)

In [183]:
map_2

### Notes
- Comm Types are Phone, SMS, and Internet
- Internet Types do not have Comm Identifiers
- Cell cgi is 'cell global identity'
- 71 unique tower locations
    - BALGOWLAH HAYES ST, 4301 times is most common
- Dataset spands 1 year, from April 1 2014 to April 1 2015
- Cell Tower Location looks to be 1:1 with Coordinates but we found 71,70,70
- All cell towers in Australia (Sydney, Melbourne, Tasmania, some inland between Sydney and Melbourne)

- Probably lives in Sydney, travels to Melbourne and Tasmania a lot based on heatmap

In [185]:
df['Comm Timedate String'].plot()

TypeError: Empty 'DataFrame': no numeric data to plot